In [1]:
import os
#IMPORT DATASET FROM KAGGLE
os.environ['KAGGLE_USERNAME'] = "rodseidel" # username from the json file
os.environ['KAGGLE_KEY'] = "6135aa1b9bebff694439825d1c78d739" # key from the json file
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.4MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 27.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.15MB/s]


In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]

In [2]:
#Alterar o código compartilhado de forma que o desempenho de classificação do RepeatedKFold tenha um score mais alto na média (pelo menos 0.85). 
#Pode ser apenas escolhendo um conjunto de parâmetros no grid search, 
#escolhendo melhor os atributos ou 
#tratanto melhor as características.

In [3]:
#MIGREI PARA O PIPELINE ESSES
#CHE
#
#Criada coluna Família (indentificado pelo sobrenome comum) - tratamento a seguir
#X["Family"] = X["Name"]

#SibSp: esposa / Parch: pais/filhos
#Informações sobre parentes e se estava sozinho ou não
#X["Relatives"] = X["SibSp"] + X["Parch"]

#for index, row in X.iterrows():
#    if row['Relatives'] > 0:
#        X.loc[index,'Alone'] = 0
#    else: X.loc[index,'Alone'] = 1

#Calcula deck a partir do número da cabine
#import re
#deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
#X['Cabin'] = X['Cabin'].fillna("U0")
#X['Deck'] = X['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
#X['Deck'] = X['Deck'].map(deck)
#X['Deck'] = X['Deck'].fillna(0)
#X['Deck'] = X['Deck'].astype(int)       

In [4]:
#MIGRADO PARA O PIPELINE
#test["Family"] = test["Name"]

#SibSp: esposa / Parch: pais/filhos
#Informações sobre parentes e se estava sozinho ou não
#test["Relatives"] = test["SibSp"] + test["Parch"]

#for index, row in test.iterrows():
#    if row['Relatives'] > 0:
#        test.loc[index,'Alone'] = 0
#    else: test.loc[index,'Alone'] = 1


#Calcula deck a partir do número da cabine
#import re
#deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
#test['Cabin'] = test['Cabin'].fillna("U0")
#test['Deck'] = test['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
#test['Deck'] = test['Deck'].map(deck)
#test['Deck'] = test['Deck'].fillna(0)
#test['Deck'] = test['Deck'].astype(int)   

In [5]:
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
from sklearn.base import BaseEstimator, TransformerMixin
import re

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

def extraiFamilia(nome):
    return nome.split(',')[0].strip()

#decks = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}

#def extraiDeck(cabin):
#    deck = re.findall("([a-zA-Z]+)", cabin)[0]
#    deck = decks.get(deck,0)
#    deck = deck if deck is not None else 0
#    return int(deck)
  
class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True, excluirFamily=True, excluirAloneAndRelativies=True):#, excluirDeck=True):
        self.excluirName = excluirName
        self.excluirFamily = excluirFamily
        self.excluirAloneAndRelativies = excluirAloneAndRelativies
        #self.excluirDeck = excluirDeck

    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Ticket', 'Cabin']
        
        #Identificação da familia
        X["Family"] = X["Name"]
        #Informações sobre parentes e se estava sozinho ou não
        X["Relatives"] = 0
        X["Alone"] = 0

        X['Cabin'] = X['Cabin'].fillna("U0")
        #X['Deck'] = 0

        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        if self.excluirFamily:
            self.colunasIndesejadas.append('Family')
        if self.excluirAloneAndRelativies:
            self.colunasIndesejadas.append('Alone')
            self.colunasIndesejadas.append('Relatives')
        #if self.excluirDeck:
        #    self.colunasIndesejadas.append('Deck')
          
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        
        if 'Alone' not in self.colunasIndesejadas:
          Xdrop["Relatives"] = Xdrop["SibSp"] + Xdrop["Parch"]
          Xdrop['Alone'] = np.where( Xdrop['Relatives'] > 0, 0, 1)

        #if 'Deck' not in self.colunasIndesejadas:
        #  Xdrop['Deck'] = Xdrop['Cabin'].apply(extraiDeck)
        
        #Coluna Cabin será excluída independentemente de qualquer tratamento
        #Xdrop = Xdrop.drop('Cabin',axis=1)

        if 'Name' not in self.colunasIndesejadas:
            Xdrop['Name'] = Xdrop['Name'].apply(extraiPronome)

            # substituir títulos com um título mais comum ou como Rare
            Xdrop['Name'] = Xdrop['Name'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                                    'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
            Xdrop['Name'] = Xdrop['Name'].replace('Mlle', 'Miss')
            Xdrop['Name'] = Xdrop['Name'].replace('Ms', 'Miss')
            Xdrop['Name'] = Xdrop['Name'].replace('Mme', 'Mrs')
            
        if 'Family' not in self.colunasIndesejadas:
            Xdrop['Family'] = Xdrop['Family'].apply(extraiFamilia)
        return Xdrop


In [7]:
#Teste
atributosDesejados = AtributosDesejados(excluirName=False,excluirFamily=False,excluirAloneAndRelativies=False)#,excluirDeck=False)
a = atributosDesejados.fit_transform(X)
a.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Family,Relatives,Alone
0,3,Mr,male,22.0,1,0,7.2500,S,Braund,1,0
1,1,Mrs,female,38.0,1,0,71.2833,C,Cumings,1,0
2,3,Miss,female,26.0,0,0,7.9250,S,Heikkinen,0,1
3,1,Mrs,female,35.0,1,0,53.1000,S,Futrelle,1,0
4,3,Mr,male,35.0,0,0,8.0500,S,Allen,0,1


In [8]:
#a.select_dtypes(include='number').columns

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [10]:
#Teste
atribNum = AtributosNumericos()
numerics = atribNum.fit_transform(X)
numerics

array([[  1.    ,   3.    ,  22.    , ...,   7.25  ,   0.    ,   0.    ],
       [  2.    ,   1.    ,  38.    , ...,  71.2833,   0.    ,   0.    ],
       [  3.    ,   3.    ,  26.    , ...,   7.925 ,   0.    ,   0.    ],
       ...,
       [889.    ,   3.    ,      nan, ...,  23.45  ,   0.    ,   0.    ],
       [890.    ,   1.    ,  26.    , ...,  30.    ,   0.    ,   0.    ],
       [891.    ,   3.    ,  32.    , ...,   7.75  ,   0.    ,   0.    ]])

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [12]:
#Teste
atribCateg = AtributosCategoricos()
teste = atribCateg.fit_transform(a)
print(atribCateg.colunasCategoricas)
print(teste)

Index(['Name', 'Sex', 'Embarked', 'Family'], dtype='object')
[['Mr' 'male' 'S' 'Braund']
 ['Mrs' 'female' 'C' 'Cumings']
 ['Miss' 'female' 'S' 'Heikkinen']
 ...
 ['Miss' 'female' 'S' 'Johnston']
 ['Mr' 'male' 'C' 'Behr']
 ['Mr' 'male' 'Q' 'Dooley']]


In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier(n_jobs=-1))
])

parametros = {
    'atributosDesejados__excluirName': [False],
    'atributosDesejados__excluirFamily': [False],
    'atributosDesejados__excluirAloneAndRelativies': [False],
    #'atributosDesejados__excluirDeck': [True],
    'classificador__max_depth': list(range(6,20,4)),
    'classificador__criterion': ['gini', 'entropy'],
    'classificador__n_estimators': [100]
}
modelo = GridSearchCV(pipetotal, param_grid=parametros, verbose=1,scoring='accuracy')

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 


(array([0.88826816, 0.83146067, 0.78651685, 0.79775281, 0.84269663,
        0.81005587, 0.84831461, 0.80337079, 0.85955056, 0.81460674,
        0.82122905, 0.85955056, 0.78089888, 0.85393258, 0.84269663,
        0.8547486 , 0.83146067, 0.84269663, 0.80898876, 0.82022472,
        0.83240223, 0.8258427 , 0.83707865, 0.83707865, 0.83146067,
        0.82681564, 0.84269663, 0.86516854, 0.80337079, 0.82022472,
        0.88826816, 0.80337079, 0.82022472, 0.84269663, 0.8258427 ,
        0.77653631, 0.85393258, 0.84831461, 0.84831461, 0.82022472,
        0.82122905, 0.86516854, 0.80337079, 0.81460674, 0.85393258,
        0.80446927, 0.83707865, 0.83707865, 0.82022472, 0.8258427 ]),
 0.830637750298161,
 0.02410014273995464)

In [ ]:
#Adicionado Tratamento específico de pronomes de tratamento     -0.77751 no kaggle
#(array([0.8603352 , 0.82022472, 0.84269663, 0.80898876, 0.82022472,
#        0.8603352 , 0.84831461, 0.7752809 , 0.8258427 , 0.8258427 ,
#        0.7877095 , 0.83146067, 0.83146067, 0.83707865, 0.78089888,
#        0.87150838, 0.80337079, 0.80337079, 0.82022472, 0.8258427 ,
#        0.81005587, 0.81460674, 0.85955056, 0.86516854, 0.83707865,
#        0.82681564, 0.83707865, 0.78651685, 0.82022472, 0.8258427 ,
#        0.82122905, 0.8258427 , 0.83707865, 0.84831461, 0.81460674,
#        0.79329609, 0.80898876, 0.85393258, 0.83707865, 0.81460674,
#        0.8547486 , 0.80337079, 0.81460674, 0.82022472, 0.83707865,
#        0.86592179, 0.83707865, 0.76404494, 0.85955056, 0.80337079]),
# 0.825578431987948,
# 0.024384320007491786)


#Adicionado True or False Name (antes estava só false)  -0.78947 no kaggle
#(array([0.82122905, 0.8258427 , 0.80898876, 0.83707865, 0.80898876,
#        0.7877095 , 0.7752809 , 0.85955056, 0.87078652, 0.8258427 ,
#        0.8547486 , 0.80898876, 0.80337079, 0.85393258, 0.83146067,
#        0.82681564, 0.83146067, 0.86516854, 0.79775281, 0.8258427 ,
#        0.84357542, 0.82022472, 0.84269663, 0.85393258, 0.78651685,
#        0.8603352 , 0.84831461, 0.83707865, 0.78651685, 0.82022472,
#        0.83240223, 0.8258427 , 0.81460674, 0.83146067, 0.80337079,
#        0.82681564, 0.84269663, 0.83146067, 0.8258427 , 0.82022472,
#        0.8547486 , 0.81460674, 0.84831461, 0.78089888, 0.83707865,
#        0.84916201, 0.83146067, 0.81460674, 0.79775281, 0.80337079]),
# 0.8261396020337705,
# 0.02267974418906704)

#Adicionado Deck  -0.78708 no kaggle
#(array([0.82122905, 0.80898876, 0.81460674, 0.83707865, 0.80898876,
#        0.83798883, 0.82022472, 0.86516854, 0.8258427 , 0.86516854,
#        0.7877095 , 0.84269663, 0.80898876, 0.84831461, 0.83146067,
#        0.81005587, 0.83707865, 0.80337079, 0.83707865, 0.84269663,
#        0.84357542, 0.85393258, 0.81460674, 0.83707865, 0.79213483,
#        0.7877095 , 0.84831461, 0.79775281, 0.8258427 , 0.80337079,
#        0.81005587, 0.81460674, 0.83707865, 0.8258427 , 0.83707865,
#        0.84916201, 0.87078652, 0.78089888, 0.84831461, 0.84269663,
#        0.80446927, 0.85955056, 0.83707865, 0.84269663, 0.81460674,
#        0.84357542, 0.87078652, 0.82022472, 0.78089888, 0.85393258]),
# 0.8280679178959263,
# 0.023137238859978912)

#Adicionado alone e relativies  -0.78468 no Kaggle
#(array([0.81564246, 0.85393258, 0.84831461, 0.79775281, 0.83146067,
#        0.82122905, 0.85393258, 0.83146067, 0.80898876, 0.84831461,
#        0.83240223, 0.83707865, 0.85393258, 0.78089888, 0.85955056,
#        0.87150838, 0.84831461, 0.80337079, 0.8258427 , 0.7752809 ,
#        0.8547486 , 0.83146067, 0.83146067, 0.84269663, 0.80337079,
#        0.83798883, 0.78651685, 0.84269663, 0.82022472, 0.85393258,
#        0.82122905, 0.79775281, 0.83707865, 0.85955056, 0.84269663,
#        0.88268156, 0.85393258, 0.8258427 , 0.78651685, 0.80898876,
#        0.83798883, 0.84269663, 0.83146067, 0.83707865, 0.81460674,
#        0.82122905, 0.84269663, 0.84831461, 0.81460674, 0.82022472]),
# 0.8306295901073381,
# 0.023172191084298816)

#ca(array([0.86592179, 0.81460674, 0.84269663, 0.81460674, 0.78651685,
#        0.80446927, 0.85955056, 0.84269663, 0.8258427 , 0.82022472,
#        0.83798883, 0.84269663, 0.84269663, 0.79213483, 0.81460674,
#        0.83240223, 0.83146067, 0.79775281, 0.83707865, 0.8258427 ,
#        0.87709497, 0.78089888, 0.82022472, 0.85955056, 0.8258427 ,
#        0.78212291, 0.82022472, 0.87078652, 0.81460674, 0.80337079,
#        0.79329609, 0.87078652, 0.84831461, 0.81460674, 0.84269663,
#        0.81005587, 0.80898876, 0.84269663, 0.84831461, 0.83707865,
#        0.8603352 , 0.82022472, 0.79213483, 0.81460674, 0.84831461,
#        0.88826816, 0.80898876, 0.82022472, 0.80898876, 0.81460674]),
# 0.8276009038980603,
# 0.025248706148010325)

In [ ]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)

In [ ]:
!kaggle competitions submit -c titanic -f submission.csv -m "new submission"